# **Сбор данных на ненецком и русском языках с сайта nvinder.ru**

## **Постановка задачи**

Для реализации модели нейронного машинного перевода и ее обучения потребуется создать собственный многоязычный параллельный корпус, так как существующие корпусы и удовлетворяющие ресурсы и материалы, связанные с ненецким языком, в сети Интернет не существуют или практически отсутствуют. Примером этого служит [опыт сбора](https://elibrary.ru/item.asp?id=65600587) текстовых данных в процессе разработки онлайн-словаря для реализации частотного словаря слов с целью точного определения объема релевантных слов в базе данных, в котором выявилось, что лингвистические материалы на ненецком языке представляют собой сложность в поиске и доступе.

К таким материалам можно отнести малосодержательные посты и комментарии в социальной сети «ВКонтакте» и достаточно крупное для НАО СМИ – общественно-политическая [газета](https://nvinder.ru/) НАО «Няръяна вындер». Впрочем, на странице сайта газеты «Ялумд’’» часто можно найти статьи на русском языке, чем на ненецком.

Помимо этого, существует большая вероятность смешать два ненецкого языка: лесной и тундровый. Настоящая работа ориентирована на литературный ненецкий язык, а именно на большеземельский говор тундрового ненецкого языка, что способствует разработке онлайн-словаря, который требуется на рынке для его сохранения и ревитализации.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Функция для получения HTML-кода страницы
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Функция для парсинга данных новости
def parse_news(news_item):
    news_link = 'https://nvinder.ru' + news_item.find('h3', class_='field-content').find('a')['href']

    # Найти ссылку "Читать далее..."
    read_more_link = 'https://nvinder.ru' + news_item.find('div', class_='readmore').find('a')['href']

    # Получаем HTML-код страницы новости
    news_page_html = get_html(read_more_link)

    if news_page_html:
        # Парсим HTML-код страницы новости
        news_page_soup = BeautifulSoup(news_page_html, 'html.parser')

        # Ищем контейнер с классом "article" и все абзацы внутри него
        article_container = news_page_soup.find('article')
        if article_container:
            paragraphs = article_container.find_all('p')

            # Инициализируем переменные для текста на разных языках и флаг для пропуска первого предложения
            example_ru = ""
            example_nn = ""
            switch_language = False
            skip_first_sentence = False

            # Извлекаем текст из каждого абзаца
            for paragraph in paragraphs:
                if "Перевод на ненецкий язык" in paragraph.get_text():
                    switch_language = True
                    skip_first_sentence = True
                    continue
                elif "Перевод на русский язык" in paragraph.get_text():
                    switch_language = False
                    skip_first_sentence = True
                    continue

                # Пропускаем первое предложение, если флаг установлен
                if skip_first_sentence:
                    skip_first_sentence = False
                    continue

                # Добавляем текст в соответствующий столбец, добавляя точку в конце, если необходимо
                text = paragraph.get_text().strip()
                if text:
                    if text[-1] not in [".", "!", "?"]:
                        text += "."
                    if switch_language:
                        example_nn += text + "\n"
                    else:
                        example_ru += text + "\n"

            # Если оба столбца не пусты, добавляем данные в список
            if example_ru.strip() and example_nn.strip():
                return {'example_ru': example_ru.strip(), 'example_nn': example_nn.strip(), 'source': news_link}
            else:
                return None
        else:
            return None
    else:
        return None

# Список для хранения данных о новостях
news_data = []

# Парсинг каждой указанной страницы и получение данных о новостях
for page_num in range(0, 1): # Необходимо указать требуемое количество страниц
    url = f'https://nvinder.ru/rubric/yalumd?page={page_num}'
    html = get_html(url)
    if html:
        soup = BeautifulSoup(html, 'html.parser')
        news_items = soup.find_all('div', class_='views-row')
        for news_item in news_items:
            parsed_news = parse_news(news_item)
            if parsed_news:
                news_data.append(parsed_news)

# Создание DataFrame для хранения данных
df = pd.DataFrame(news_data)

# Вывод первых нескольких строк DataFrame
print(df.head())

# Сохранение данных в CSV файл
df.to_csv('news_data.csv', index=False)

                                          example_ru  \
0  В последний день января стартовал новый проект...   
1  Рассказываем о потомственных оленеводах из Инд...   
2  В этом году Институт народов Севера РГПУ имени...   
3  В Нарьян-Маре подвели итоги конкурса «Семейно-...   
4  В Ненецкой окружной библиотеке прошёл краеведч...   

                                          example_nn  \
0  Хаювы ирий’ пудана яляхана этнокультурной цент...   
1  Тюку яля’ вадими’ Индига’ ӈэсы’ тер’’ тыбэрма’...   
2  Тюку по’ А.И.Герцен’ нювм’ нюбета РГПУ’ ӈэрм’ ...   
3  Тюку по’ нертей яля’’ пиркана Нарьяна маркы’ а...   
4  Тюку по’ я’ сяр нина’’ илена’’ хибяри’’ ханяри...   

                                              source  
0  https://nvinder.ru/article/vypusk-no-12-21642-...  
1  https://nvinder.ru/article/vypusk-no-10-21640-...  
2  https://nvinder.ru/article/vypusk-no-9-21639-o...  
3  https://nvinder.ru/article/vypusk-no-7-21637-o...  
4  https://nvinder.ru/article/vypusk-no-6-21636-o..

In [5]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

# Загрузка данных из CSV файла в датафрейм
df = pd.read_csv('news_data.csv')

# Функция для разделения текста на предложения
def split_text_into_sentences(text):
    return sent_tokenize(text)

# Создание нового датафрейма для разделенных записей
split_news_data = pd.DataFrame(columns=['source', 'example_ru', 'example_nn'])

# Создание нового датафрейма для записей, которые не удалось разделить
row_news_data = pd.DataFrame(columns=['source', 'example_ru', 'example_nn'])

# Перебор всех записей в исходном датафрейме
for idx, row in df.iterrows():
    example_ru_sentences = split_text_into_sentences(row['example_ru'])
    example_nn_sentences = split_text_into_sentences(row['example_nn'])

    # Проверка на равенство количества предложений
    if len(example_ru_sentences) == len(example_nn_sentences):
        # Добавление разделенных записей в новый датафрейм
        for i in range(len(example_ru_sentences)):
            split_news_data = pd.concat([split_news_data, pd.DataFrame({
                'source': row['source'],
                'example_ru': [example_ru_sentences[i]],
                'example_nn': [example_nn_sentences[i]]
            })], ignore_index=True)
    else:
        print(f"Index {idx} has unequal number of sentences")
        # Добавление исходной записи в новый датафрейм
        row_news_data = pd.concat([row_news_data, pd.DataFrame({
            'source': [row['source']],
            'example_ru': [row['example_ru']],
            'example_nn': [row['example_nn']]
        })], ignore_index=True)

# Сохранение разделенных записей в CSV файл
split_news_data.to_csv('split_news_data.csv', index=False)

# Сохранение записей с неравным количеством предложений в CSV файл
row_news_data.to_csv('row_news_data.csv', index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Index 0 has unequal number of sentences
Index 1 has unequal number of sentences
Index 2 has unequal number of sentences
Index 3 has unequal number of sentences
Index 4 has unequal number of sentences
Index 5 has unequal number of sentences
Index 6 has unequal number of sentences
Index 7 has unequal number of sentences
Index 8 has unequal number of sentences
Index 9 has unequal number of sentences
Index 11 has unequal number of sentences
Index 12 has unequal number of sentences
Index 13 has unequal number of sentences
Index 14 has unequal number of sentences


In [7]:
# Функция для подсчета количества предложений в тексте
def count_sentences(text):
    if pd.isna(text):  # Проверка на NaN
        return 0
    return len(sent_tokenize(text))

# Функция для подсчета количества предложений в каждой записи
def count_sentences_per_record(row):
    count_ru_sentences = count_sentences(row['example_ru'])
    count_nn_sentences = count_sentences(row['example_nn'])
    return pd.Series({'sentences_ru': count_ru_sentences, 'sentences_nn': count_nn_sentences})

# Создание новых столбцов с количеством предложений для каждой записи
sentence_counts = df.apply(count_sentences_per_record, axis=1)

# Объединение результатов с исходным датафреймом
df_with_sentence_counts = pd.concat([df, sentence_counts], axis=1)

# Вывод количества предложений в красивом формате
for idx, row in df_with_sentence_counts.iterrows():
    print(f"Запись {idx}:")
    print(f"Количество предложений в столбце 'example_ru': {row['sentences_ru']}")
    print(f"Количество предложений в столбце 'example_nn': {row['sentences_nn']}\n")

Запись 0:
Количество предложений в столбце 'example_ru': 37
Количество предложений в столбце 'example_nn': 45

Запись 1:
Количество предложений в столбце 'example_ru': 43
Количество предложений в столбце 'example_nn': 44

Запись 2:
Количество предложений в столбце 'example_ru': 39
Количество предложений в столбце 'example_nn': 37

Запись 3:
Количество предложений в столбце 'example_ru': 40
Количество предложений в столбце 'example_nn': 46

Запись 4:
Количество предложений в столбце 'example_ru': 33
Количество предложений в столбце 'example_nn': 35

Запись 5:
Количество предложений в столбце 'example_ru': 34
Количество предложений в столбце 'example_nn': 44

Запись 6:
Количество предложений в столбце 'example_ru': 35
Количество предложений в столбце 'example_nn': 41

Запись 7:
Количество предложений в столбце 'example_ru': 36
Количество предложений в столбце 'example_nn': 43

Запись 8:
Количество предложений в столбце 'example_ru': 22
Количество предложений в столбце 'example_nn': 28

З

Эмпирически установленный факт частого расхождения в количестве предложений между русскоязычными и ненецкоязычными текстами указывает на возможные проблемы с качеством перевода. Это несоответствие может свидетельствовать о пропущенных, упрощенных или измененных фрагментах, что снижает надежность данных для обучения модели нейронного машинного перевода.

Для более обоснованных выводов необходимо провести качественный анализ перевода, например, выборочную проверку соответствия оригинального и переведенного текстов. Поскольку для ненецкого языка не существуют готовые решения в области машинного перевода, в том числе автоматизированные метрики качества (такие как BLEU, TER или METEOR), альтернативным подходом может стать экспертная оценка и частотный анализ текстов.